In [1]:
!pip install tensorflow==2.1
!pip install -U efficientnet

     |████████████████████████████████| 421.8 MB 18 kB/s 
     |████████████████████████████████| 3.8 MB 37.6 MB/s 
     |████████████████████████████████| 448 kB 30.9 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=8a750aa5b72279f5a35557c05df5001de67eec93e2a843765369846608c2ad6a
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Attempting uninstall: gast
    Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.2.0
    Uninstall

In [2]:
import math, re, os, random
import tensorflow as tf
import numpy as np
import efficientnet.tfkeras as efn
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.1.0


In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470
REPLICAS:  8


In [4]:
USE_FLOAT16=False
XLA_ACCELERATE=False
if USE_FLOAT16:
    from tensorflow.keras.mixed_precision import experimental as mixed_precision
    if tpu: 
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_bfloat16')
    else: 
        policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    print('Mixed precision enabled')

if XLA_ACCELERATE:
    tf.config.optimizer.set_jit(True)
    print('Accelerated Linear Algebra enabled')

In [5]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
PATH2 = KaggleDatasets().get_gcs_path('tf-flower-photo-tfrec')

In [6]:
IMAGE_SIZE = [224, 224]
EPOCHS = 10
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
img_size=224
GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
TRAINING_FILENAMES += tf.io.gfile.glob(PATH2 + '/imagenet/tfrecords-jpeg-224x224/*.tfrec')
TRAINING_FILENAMES += tf.io.gfile.glob(PATH2 + '/inaturalist_1/tfrecords-jpeg-224x224/*.tfrec')
TRAINING_FILENAMES += tf.io.gfile.glob(PATH2 + '/openimage/tfrecords-jpeg-224x224/*.tfrec')
TRAINING_FILENAMES += tf.io.gfile.glob(PATH2 + '/oxford_102/tfrecords-jpeg-224x224/*.tfrec')
TRAINING_FILENAMES += tf.io.gfile.glob(PATH2 + '/tf_flowers/tfrecords-jpeg-224x224/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')
TRAINING_FILENAMES=TRAINING_FILENAMES+VALIDATION_FILENAMES        

In [7]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'] 

In [8]:
def random_blockout(img, sl=0.1, sh=0.2, rl=0.4):
    p=random.random()
    if p>=0.25:
        h, w, c = img_size, img_size, 3
        origin_area = tf.cast(h*w, tf.float32)

        e_size_l = tf.cast(tf.round(tf.sqrt(origin_area * sl * rl)), tf.int32)
        e_size_h = tf.cast(tf.round(tf.sqrt(origin_area * sh / rl)), tf.int32)

        e_height_h = tf.minimum(e_size_h, h)
        e_width_h = tf.minimum(e_size_h, w)

        erase_height = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_height_h, dtype=tf.int32)
        erase_width = tf.random.uniform(shape=[], minval=e_size_l, maxval=e_width_h, dtype=tf.int32)

        erase_area = tf.zeros(shape=[erase_height, erase_width, c])
        erase_area = tf.cast(erase_area, tf.uint8)

        pad_h = h - erase_height
        pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
        pad_bottom = pad_h - pad_top

        pad_w = w - erase_width
        pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
        pad_right = pad_w - pad_left

        erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
        erase_mask = tf.squeeze(erase_mask, axis=0)
        erased_img = tf.multiply(tf.cast(img,tf.float32), tf.cast(erase_mask, tf.float32))

        return tf.cast(erased_img, img.dtype)
    else:
        return tf.cast(img, img.dtype)

In [9]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) 
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label 

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "id": tf.io.FixedLenFeature([], tf.string),  
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

In [10]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image= random_blockout(image)
    return image, label   

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

In [11]:
NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Dataset: 70470 training images, 3712 validation images, 7382 unlabeled test images


In [12]:
def lrfn(epoch):
    LR_START = 0.00001
    LR_MAX = 0.00005 * strategy.num_replicas_in_sync
    LR_MIN = 0.00001
    LR_RAMPUP_EPOCHS = 5
    LR_SUSTAIN_EPOCHS = 0
    LR_EXP_DECAY = .8
    
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)

In [13]:
def get_model():
    model_input = tf.keras.Input(shape=[*IMAGE_SIZE, 3], name="input")

    dummy = tf.keras.layers.Lambda(lambda x: x)(model_input)
    
    outputs = []
    
    x = efn.EfficientNetB0(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
    
    x = efn.EfficientNetB1(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    x = efn.EfficientNetB2(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    x = efn.EfficientNetB3(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    x = efn.EfficientNetB4(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    x = efn.EfficientNetB5(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    x = efn.EfficientNetB6(include_top=False, weights='noisy-student', input_shape=[*IMAGE_SIZE, 3], pooling='avg')(dummy)
    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    outputs.append(x)
        
    model = tf.keras.Model(model_input, outputs, name='mega-efnnet')
    model.summary()
    return model

In [14]:
def create_model():
    with strategy.scope():
        model = get_model()

    with strategy.scope():    
        model.compile(
            optimizer = "adam",
            loss = [tf.keras.losses.SparseCategoricalCrossentropy(), 
                    tf.keras.losses.SparseCategoricalCrossentropy(), 
                    tf.keras.losses.SparseCategoricalCrossentropy(), 
                    tf.keras.losses.SparseCategoricalCrossentropy(), 
                    tf.keras.losses.SparseCategoricalCrossentropy(), 
                    tf.keras.losses.SparseCategoricalCrossentropy(),  
                    tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics=["accuracy"]           
        )
    return model

In [15]:
def train(model, ds_train, stepsTrain, ds_val, stepsVal=0):
    callbacks = [lr_callback]
        
    history = model.fit(ds_train, 
                        validation_data=ds_val, 
                        verbose = 1,
                        steps_per_epoch=stepsTrain, 
                        validation_steps=stepsVal, 
                        epochs=5,
                        callbacks = callbacks)
    
    return history

In [16]:
ds_train = get_training_dataset().map(lambda img, label: (img, (label, label, label, label, label, label, label)))
ds_val = get_validation_dataset().map(lambda img, label: (img, (label, label, label, label, label, label, label)))
training_steps = count_data_items(TRAINING_FILENAMES) / (BATCH_SIZE * strategy.num_replicas_in_sync)

In [17]:
model = create_model()

165232640/165226952 [==============================] - 2s 0us/step
Model: "mega-efnnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 224, 224, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
efficientnet-b0 (Model)         (None, 1280)         4049564     lambda[0][0]                     
__________________________________________________________________________________________________
efficientnet-b1 (Model)         (None, 1280)         6575232     lambda[0][0]                     
_____________________

In [18]:
# train(model, ds_train, training_steps, ds_val)

In [19]:
callbacks = [lr_callback]
        
history = model.fit(ds_train, 
                    validation_data=ds_val, 
                    verbose = 1,
                    steps_per_epoch=training_steps, 
                    validation_steps=0, 
                    epochs=5,
                    callbacks = callbacks)

Train for 68.818359375 steps

Epoch 00001: LearningRateScheduler reducing learning rate to 1e-05.
Epoch 1/5
69/68 [==============================] - 1261s 18s/step - loss: nan - dense_loss: nan - dense_1_loss: nan - dense_2_loss: nan - dense_3_loss: nan - dense_4_loss: nan - dense_5_loss: nan - dense_6_loss: nan - dense_accuracy: 0.0224 - dense_1_accuracy: 0.0224 - dense_2_accuracy: 0.0221 - dense_3_accuracy: 0.0224 - dense_4_accuracy: 0.0222 - dense_5_accuracy: 0.0222 - dense_6_accuracy: 0.0221 - val_loss: nan - val_dense_loss: nan - val_dense_1_loss: nan - val_dense_2_loss: nan - val_dense_3_loss: nan - val_dense_4_loss: nan - val_dense_5_loss: nan - val_dense_6_loss: nan - val_dense_accuracy: 0.0213 - val_dense_1_accuracy: 0.0213 - val_dense_2_accuracy: 0.0213 - val_dense_3_accuracy: 0.0213 - val_dense_4_accuracy: 0.0213 - val_dense_5_accuracy: 0.0213 - val_dense_6_accuracy: 0.0213
